<a href="https://colab.research.google.com/github/RocioUrquijo/Practica3RocioUrquijo/blob/main/Practica3_RocioUrquijo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalamos librerias

In [1]:
!pip install datasets evaluate transformers[sentencepiece] -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.6 MB/s eta 0:00:00


Copiamos nuestro token generado

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Cambiamos al dataset llamado: Hello-SimpleAI/HC3 con el subset reddit_eli5

In [3]:
from datasets import load_dataset
raw_dataset = load_dataset("papluca/language-identification")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/papluca___csv/papluca--language-identification-ad5bdc8c9b1a4985/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Vemos nuestro dataset elegido:

In [4]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'text'],
        num_rows: 70000
    })
    test: Dataset({
        features: ['labels', 'text'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['labels', 'text'],
        num_rows: 10000
    })
})

In [5]:
raw_dataset['train']

Dataset({
    features: ['labels', 'text'],
    num_rows: 70000
})

In [6]:
raw_dataset['train'].to_pandas()

,labels,text
0,pt,"os chefes de defesa da estónia, letónia, lituâ..."
1,bg,размерът на хоризонталната мрежа може да бъде ...
2,zh,很好，以前从不去评价，不知道浪费了多少积分，现在知道积分可以换钱，就要好好评价了，后来我就把...
3,th,สำหรับ ของเก่า ที่ จริงจัง ลอง honeychurch ...
4,ru,Он увеличил давление .
...,...,...
69995,ja,本格的なゲーミングヘッドホンでした。 今まで使ってた1万円するパナソニックのヘッドホンは何だ...
69996,el,"Ναι , ξέρω ένα που είναι ακόμα έτσι , αλλά αυτ..."
69997,ur,اور مجھے اس ملک کے بارے میں معلوم نہیں ہے کہ گ...
69998,es,Se me rompió uno al sacarlo del cargador. Cali...


Cargamos el modelo preentrenado

In [7]:
from transformers import AutoTokenizer, DataCollatorWithPadding

model_checkpoint = "papluca/xlm-roberta-base-language-detection"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


Tokenizamos la columna que nos interesa: review

In [8]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

In [9]:
tokenized_dataset = raw_dataset.map(tokenize_function, batched=True)
tokenized_dataset

  0%|          | 0/70 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 70000
    })
    test: Dataset({
        features: ['labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 10000
    })
})

In [10]:
tokenized_dataset = tokenized_dataset.rename_column('labels','label')
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 70000
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 10000
    })
})

In [12]:
from datasets import DatasetDict,Dataset
# 1. Revolvemos el dataset con el método shuffle
new_tokenized_dataset = tokenized_dataset["train"].shuffle()
# 2. Calculamos el número de elementos del dataset
len_dataset = len(tokenized_dataset["train"])
# 3. Partimos el dataset en dos trozos
train_dataset = new_tokenized_dataset[0:int(len_dataset*0.8)]
test_dataset = new_tokenized_dataset[int(len_dataset*0.8):]
new_dataset = DatasetDict({"train":Dataset.from_dict(train_dataset),"test":Dataset.from_dict(test_dataset)})


In [13]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
from transformers import TrainingArguments
training_args = TrainingArguments("clasificador-languagedetection",evaluation_strategy="epoch")

In [15]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=20)

In [16]:
import evaluate
import numpy as np

def compute_metrics(eval_preds):
  metric = evaluate.load("accuracy")
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

In [17]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=new_dataset["train"],
    eval_dataset=new_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [18]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 56000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 21000
  Number of trainable parameters = 278059028
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster 

ValueError: ignored

In [19]:
# Vamos a la carpeta donde se ha guardado nuestro modelo, es el valor que
# definimos previamente en el objeto TrainingArguments
%cd clasificador-languagedetection
# Subimos el modelo indicando un mensaje de confirmación, y una etiqueta. 
trainer.push_to_hub(commit_message="Training complete", tags="classification")

/content/clasificador-languagedetection


Cloning https://huggingface.co/RocioUrquijo/clasificador-languagedetection into local empty directory.
Saving model checkpoint to clasificador-languagedetection
Configuration saved in clasificador-languagedetection/config.json
Model weights saved in clasificador-languagedetection/pytorch_model.bin
tokenizer config file saved in clasificador-languagedetection/tokenizer_config.json
Special tokens file saved in clasificador-languagedetection/special_tokens_map.json
Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.


Upload file pytorch_model.bin:   0%|          | 32.0k/1.04G [00:00<?, ?B/s]

Upload file sentencepiece.bpe.model:   1%|          | 32.0k/4.83M [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 3.43k/3.43k [00:00<?, ?B/s]

Upload file tokenizer.json:   0%|          | 32.0k/16.3M [00:00<?, ?B/s]

remote: Scanning LFS files for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/RocioUrquijo/clasificador-languagedetection
   c60bfce..baa5916  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/RocioUrquijo/clasificador-languagedetection
   c60bfce..baa5916  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}}
To https://huggingface.co/RocioUrquijo/clasificador-languagedetection
   baa5916..45f7acc  main -> main

   baa5916..45f7acc  main -> main



'https://huggingface.co/RocioUrquijo/clasificador-languagedetection/commit/baa59167cc6506940adca3d6973c26272c0833e2'

In [20]:
from transformers import pipeline
classifier = pipeline('text-classification', model='RocioUrquijo/clasificador-languagedetection')

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--RocioUrquijo--clasificador-languagedetection/snapshots/45f7acc6a63b985a41a5af20a0221deaacf9ad9b/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "RocioUrquijo/clasificador-languagedetection",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "ja",
    "1": "nl",
    "2": "ar",
    "3": "pl",
    "4": "de",
    "5": "it",
    "6": "pt",
    "7": "tr",
    "8": "es",
    "9": "hi",
    "10": "el",
    "11": "ur",
    "12": "bg",
    "13": "en",
    "14": "fr",
    "15": "zh",
    "16": "ru",
    "17": "th",
    "18": "sw",
    "19": "vi"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "ar": 2,
    "bg": 12,
    "de": 4,
    "

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--RocioUrquijo--clasificador-languagedetection/snapshots/45f7acc6a63b985a41a5af20a0221deaacf9ad9b/pytorch_model.bin
All model checkpoint weights were used when initializing XLMRobertaForSequenceClassification.

All the weights of XLMRobertaForSequenceClassification were initialized from the model checkpoint at RocioUrquijo/clasificador-languagedetection.
If your task is similar to the task the model of the checkpoint was trained on, you can already use XLMRobertaForSequenceClassification for predictions without further training.


loading file sentencepiece.bpe.model from cache at /root/.cache/huggingface/hub/models--RocioUrquijo--clasificador-languagedetection/snapshots/45f7acc6a63b985a41a5af20a0221deaacf9ad9b/sentencepiece.bpe.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--RocioUrquijo--clasificador-languagedetection/snapshots/45f7acc6a63b985a41a5af20a0221deaacf9ad9b/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--RocioUrquijo--clasificador-languagedetection/snapshots/45f7acc6a63b985a41a5af20a0221deaacf9ad9b/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--RocioUrquijo--clasificador-languagedetection/snapshots/45f7acc6a63b985a41a5af20a0221deaacf9ad9b/tokenizer_config.json


In [21]:
classifier('Nice assignment, really helpful.')

[{'label': 'en', 'score': 0.9942048192024231}]

In [22]:
classifier('Une phrase dans une autre langue.')

[{'label': 'fr', 'score': 0.9948728680610657}]

In [23]:
classifier('Das haus ist rot.')

[{'label': 'de', 'score': 0.99411940574646}]

In [24]:
classifier('Tiene mucha precision.')

[{'label': 'es', 'score': 0.9923408031463623}]